In [39]:
import  torch
from torch_geometric.data import Data

In [14]:
import  pickle 

from    src.gcn import GCN

import  torch.nn.functional as F
import  tqdm

In [36]:
data_path   = './data/m2skg/'
filename    = 'assertion_train.pkl'

node_embed_dim  = 100

In [13]:
with open(data_path+filename,'rb') as f:
    data = pickle.load(f)

In [58]:
types

{'SubjectPseudoIdentifier': 0,
 'Code': 1,
 'DrugPrescription': 2,
 'LabTestEvent': 3,
 'AdministrativeCase': 4,
 'OrganSupport': 5}

In [40]:
entities= {e : i for i,e in enumerate( list(set([s for s,t in data])))}
types   = {t : i for i,t in enumerate( list(set([t for s,t in data])))}

type_list = {key : []  for key in types.keys()}

# create an edge between all nodes of the same type
entity_types = []
for i,(entity,type) in enumerate(data): 
    type_list[type] += [entity]
    entity_types += [types[type]]


edges = []

for key,entity_list in type_list.items():

    for source in entity_list:
        for target in entity_list:
            if source != target:
                edges.append( [  entities[source] , entities[target]  ]   )

edges = torch.tensor(edges).T


In [50]:
pyg_df = Data(x = torch.nn.Embedding(len(entities), node_embed_dim), edge_index = edges, y = torch.tensor(entity_types)  )

In [51]:
pyg_df

Data(x=Embedding(1253, 100), edge_index=[2, 463624], y=[1253])

In [80]:
model = GCN(node_embed_dim,hid_dim=100,num_classes=len(types))
print(model)

GCN(
  (conv1): GCNConv(100, 100)
  (conv2): GCNConv(100, 100)
  (classifier): Linear(in_features=100, out_features=6, bias=True)
)


In [81]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [82]:
epochs  = 100

for i in tqdm.tqdm(range(epochs)):

    model.train()
    optimizer.zero_grad()

    #embeds = model(node_embeds, edge_index)
    out,emb_out = model(pyg_df.x.weight,pyg_df.edge_index)

    #loss =  criterion(out, data.y)
    loss = criterion(out , pyg_df.y)

    acc  = (F.softmax(out,dim=1).argmax(dim=1) == pyg_df.y).numpy().sum() / len(pyg_df.y)

    loss.backward()
    optimizer.step()

    print(f'loss {loss.item():0.02f} acc {acc:0.02f}')

  1%|          | 1/100 [00:00<00:41,  2.38it/s]

loss 1.78 acc 0.32


  2%|▏         | 2/100 [00:00<00:40,  2.44it/s]

loss 1.64 acc 0.49


  3%|▎         | 3/100 [00:01<00:39,  2.44it/s]

...

loss 1.49 acc 0.49


 99%|█████████▉| 99/100 [00:40<00:00,  2.41it/s]

loss 1.49 acc 0.49


100%|██████████| 100/100 [00:41<00:00,  2.43it/s]

loss 1.49 acc 0.49
